<h1><center>Введение в анализ данных</center></h1>
<hr>
<h2><center>Rossman Store Sales</center></h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

<h2>Подготовка данных:</h2>

<h3>Основная таблица</h3>

In [ ]:
filepath = '../input/train.csv'
df = pd.read_csv(filepath, sep=',')
df.head()

Заменим в StateHoliday буквы на цифры

In [ ]:
df = df.replace({'StateHoliday': {'0': 0, 'a': 1, 'b': 2, 'c': 3}})

<h3>Дополнительная таблица</h3>

In [ ]:
filepath = '../input/store.csv'
df_store = pd.read_csv(filepath, sep=',')
df_store.head()

Заменяем в категориальных переменных значения на цифры

In [ ]:
df_store = df_store.replace({'StoreType': {'a': 0, 'b': 1, 'c': 2, 'd': 3}})
df_store = df_store.replace({'Assortment': {'a': 0, 'b': 1, 'c': 2}})

Объединяем две таблицы

In [ ]:
df_full = df.merge(df_store, on='Store', how='left')
df_full.head()

<h3>Тестовые данные</h3>

In [ ]:
filepath = '../input/test.csv'
df_test = pd.read_csv(filepath, sep=',').set_index('Id')
df_test.head()

In [ ]:
df_test = df_test.replace({'StateHoliday': {'0': 0, 'a': 1, 'b': 2, 'c': 3}})

В тестовых данных есть 11 объектов, у которых в столбце Open стоит nan. Заменим их на 1, и приведем весь столбец к целочисленному формату.

In [ ]:
df_test.isnull().loc[:,'Open'].value_counts()

Заполняем пропуски в Open единицами (пропусков всего 11, так что это незначительно)

In [ ]:
df_test.Open.fillna(value=1, inplace=True)
df_test['Open'] = df_test['Open'].astype(int)
df_test['Open'].unique()

Присоединяем дополнительную таблицу

In [ ]:
df_test_full = df_test.merge(df_store, on='Store', how='left')
df_test_full.head()

In [ ]:
del df
del df_store
del df_test

<h2>Формат выходных данных</h2>

In [ ]:
filepath = '../input/sample_submission.csv'
df_out = pd.read_csv(filepath, sep=',')
print(len(df_out))
df_out.head()

<h2>Random Forest</h2>

Попробуем лучше представить данные - вытащим дату, месяц, год

In [ ]:
#train
df_full['Year'] = df_full['Date'].apply(lambda x: int(x[:4]))
df_full['Month'] = df_full['Date'].apply(lambda x: int(x[5:7]))
df_full['Day'] = df_full['Date'].apply(lambda x: int(x[8:10]))

#test
df_test_full['Year'] = df_test_full['Date'].apply(lambda x: int(x[:4]))
df_test_full['Month'] = df_test_full['Date'].apply(lambda x: int(x[5:7]))
df_test_full['Day'] = df_test_full['Date'].apply(lambda x: int(x[8:10]))

Введем переменную CompetitionOpen - открыт ли сейчас магазин конкурентов

In [ ]:
df_full['CompetitionOpen'] = ((df_full['CompetitionOpenSinceYear'] < df_full['Year']) |\
                             ((df_full['CompetitionOpenSinceYear'] == df_full['Year']) &\
                              (df_full['CompetitionOpenSinceMonth'] <= df_full['Month']))).astype(int)

df_test_full['CompetitionOpen'] = ((df_test_full['CompetitionOpenSinceYear'] < df_test_full['Year']) |\
                             ((df_test_full['CompetitionOpenSinceYear'] == df_test_full['Year']) &\
                              (df_test_full['CompetitionOpenSinceMonth'] <= df_test_full['Month']))).astype(int)

Полная дата и номер недели в году

In [ ]:
df_full['FullDate'] = pd.to_datetime(df_full.Date, format='%Y-%m-%d')
df_test_full['FullDate'] = pd.to_datetime(df_test_full.Date, format='%Y-%m-%d')

import datetime
df_full['WeekNum'] = df_full['FullDate'].apply(func=lambda x: x.isocalendar()[1])
df_test_full['WeekNum'] = df_test_full['FullDate'].apply(func=lambda x: x.isocalendar()[1])

Средний чек в каждом магазине за все время наблюдений

In [ ]:
sum_sales = df_full.groupby('Store')['Sales'].sum()
sum_customers = df_full.groupby('Store')['Customers'].sum()
mean_check = sum_sales/sum_customers
df_meancheck = pd.DataFrame(mean_check, columns=['MeanCheck'])
df_meancheck = df_meancheck.reset_index()
df_full = df_full.merge(df_meancheck, on='Store', how='left')
df_test_full = df_test_full.merge(df_meancheck, on='Store', how='left')
del df_meancheck, sum_sales, sum_customers, mean_check

Действует ли в данный день акция Promo2 в этом магазине

In [ ]:
df_full['PromoInterval'].unique()

In [ ]:
intervs_list = {'Jan,Apr,Jul,Oct': (1,4,7,10), 'Feb,May,Aug,Nov': (2,5,8,11), 'Mar,Jun,Sept,Dec': (3,6,9,12), np.nan: ()}

#train
df_full['Promo2Now'] = map(lambda week, month, year, promo2, interv, since_week, since_year:
                          int((promo2) == 1 & ((year > since_year) | ((year == since_year) & (week >= since_week))) &\
                           month in intervs_list[interv]), 
                           df_full['WeekNum'], df_full['Month'], df_full['Year'], df_full['Promo2'],
                           df_full['PromoInterval'], df_full['Promo2SinceWeek'], df_full['Promo2SinceYear'])

#test
df_test_full['Promo2Now'] = map(lambda week, month, year, promo2, interv, since_week, since_year:
                          int((promo2) == 1 & ((year > since_year) | ((year == since_year) & (week >= since_week))) &\
                           month in intervs_list[interv]), 
                           df_test_full['WeekNum'], df_test_full['Month'], df_test_full['Year'], 
                           df_test_full['Promo2'], df_test_full['PromoInterval'], df_test_full['Promo2SinceWeek'], 
                           df_test_full['Promo2SinceYear'])

In [ ]:
df_full.head().T

In [ ]:
columns = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceYear', 'Year', 'Month', 'Day',
           'WeekNum', 'MeanCheck']
X = np.array(df_full.loc[:,columns]).astype(int)
y = np.array(df_full.loc[:,'Sales']).astype(int)
X_test = np.array(df_test_full.loc[:,columns]).astype(int)

Можно также попробовать в качестве целевой переменной взять количество покупателей, а потом его умножать на средний чек

In [ ]:
#попробуем предсказывать число покупателей, а прибыль получать из среднего чека
columns = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
           'CompetitionDistance', 'CompetitionOpenSinceYear', 'Year', 'Month', 'Day',
           'WeekNum', 'MeanCheck']
X = np.array(df_full.loc[:,columns]).astype(int)
y = np.array(df_full.loc[:,'Customers']).astype(int)
X_test = np.array(df_test_full.loc[:,columns]).astype(int)

In [ ]:
cls = RandomForestRegressor(n_estimators=30, criterion='mse').fit(X, y)

In [ ]:
score = cls.predict(X_test)

In [ ]:
score_final = score*df_test_full.MeanCheck

In [ ]:
features = {}
for i in range(len(columns)):
    features[columns[i]] = cls.feature_importances_[i]

Важность признаков в порядке убывания

In [ ]:
import operator
sorted_features = sorted(features.items(), key=operator.itemgetter(1), reverse=True)
sorted_features

**Лучшая точность:**<br>
<br>
0.15051<br>
0.12415<br><br>
Получилось, что лучше предсказывать число покупателей. Если предсказывать напрямую прибыль, то лучше 0,13 не получалось

<h2>Boosting</h2>

In [ ]:
#%%timeit
#boost = GradientBoostingClassifier(n_estimators=50).fit(X[:100], y[:100])

Очень долго работает - с учетом асимптотики на всей выборке будет обучаться больше года! 

<h2>Линейная регрессия</h2>

Векторизуем остальные категориальные признаки: DayOfWeek, StoreType, Assortment, и нормализуем вещественный CompetitionDistance

In [ ]:
df_full_linear = df_full.copy()
day_of_week = OneHotEncoder().fit_transform(df_full_linear['DayOfWeek'].values.reshape(-1, 1)).toarray().astype(int)
store_type = OneHotEncoder().fit_transform(df_full_linear['StoreType'].values.reshape(-1, 1)).toarray().astype(int)
assortment = OneHotEncoder().fit_transform(df_full_linear['Assortment'].values.reshape(-1, 1)).toarray().astype(int)

for i in range(len(day_of_week[0])):
    df_full_linear['day_of_week_{}'.format(i)] = day_of_week[:,i]
    
for i in range(len(store_type[0])):
    df_full_linear['store_type_{}'.format(i)] = store_type[:,i]
    
for i in range(len(assortment[0])):
    df_full_linear['assortment_{}'.format(i)] = assortment[:,i]
    
df_full_linear.drop(df_full_linear[['DayOfWeek', 'StoreType', 'Assortment']], axis=1, inplace=True)

In [ ]:
df_full_linear.head().T

In [ ]:
df_full_linear.info()

При обучении будем учитывать только открытые магазины, где выручка не равна нулю

In [ ]:
columns = ['Store', 'Promo', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance', 'Promo2',
          'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
          'day_of_week_6', 'store_type_0', 'store_type_1', 'store_type_2', 'store_type_3',
          'assortment_0', 'assortment_1', 'assortment_2', 'MeanCheck']
X_linear = np.array(df_full_linear.loc[df_full_linear.Open == 1,columns])
y_linear = np.array(df_full_linear.loc[df_full_linear.Open == 1,'Sales'])

In [ ]:
X_linear.shape

Избавимся от Nan в CompetitionDistance - заполним средним значением

In [ ]:
np.unique(np.isnan(X_linear[:,4]))

In [ ]:
X_linear[np.isnan(X_linear[:,4]),4] = 0

In [ ]:
np.unique(np.isnan(X_linear[:,4]))

Отнормируем данные

In [ ]:
X_linear = StandardScaler().fit_transform(X_linear)
#y_linear = (y_linear - y_linear.mean())/y_linear.std()

In [ ]:
y_linear[0]

Аналогично преобразуем тестовые данные

In [ ]:
df_linear_test = df_test_full.copy()
day_of_week = OneHotEncoder().fit_transform(df_linear_test['DayOfWeek'].values.reshape(-1, 1)).toarray().astype(int)
store_type = OneHotEncoder().fit_transform(df_linear_test['StoreType'].values.reshape(-1, 1)).toarray().astype(int)
assortment = OneHotEncoder().fit_transform(df_linear_test['Assortment'].values.reshape(-1, 1)).toarray().astype(int)

for i in range(len(day_of_week[0])):
    df_linear_test['day_of_week_{}'.format(i)] = day_of_week[:,i]
    
for i in range(len(store_type[0])):
    df_linear_test['store_type_{}'.format(i)] = store_type[:,i]
    
for i in range(len(assortment[0])):
    df_linear_test['assortment_{}'.format(i)] = assortment[:,i]
    
df_linear_test.drop(df_linear_test[['DayOfWeek', 'StoreType', 'Assortment']], axis=1, inplace=True)

df_linear_test.loc[:, 'CompetitionDistance'] = (df_linear_test.loc[:, 'CompetitionDistance'] \
                                                - df_linear_test.loc[:, 'CompetitionDistance'].mean()).div(\
                                                 df_linear_test.loc[:, 'CompetitionDistance'].std())

columns = ['Store', 'Promo', 'StateHoliday', 'SchoolHoliday', 'CompetitionDistance', 'Promo2',
          'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
          'day_of_week_6', 'store_type_0', 'store_type_1', 'store_type_2', 'store_type_3',
          'assortment_0', 'assortment_1', 'assortment_2', 'MeanCheck']
X_linear_test = np.array(df_linear_test.loc[:,columns])

X_linear_test[np.isnan(X_linear_test[:,4]),4] = 0
X_linear_test = StandardScaler().fit_transform(X_linear_test)

In [ ]:
lin_reg = LinearRegression().fit(X_linear, y_linear)

In [ ]:
ridge = Ridge(alpha=1.0).fit(X_linear, y_linear)

Очень плохой R^2, линейная регрессия не подходит

In [ ]:
lin_reg.score(X_linear, y_linear)

In [ ]:
ridge.score(X_linear, y_linear)

In [ ]:
score = ridge.predict(X_linear_test)

In [ ]:
ridge.coef_

Для закрытых магазинов заполним выручку нулями

In [ ]:
score[X_linear_test[:,1] == 0] = 0

Лучшая точность:<br>
<br>
0.43265<br>
0.43062<br><br>
Линейная регрессия плохо подходит для этой задачи. Можно было бы еще попробовать с весами - чем более старые данные, тем меньше вес

<h2>SVM</h2>

In [ ]:
#%%timeit
#svr = SVR().fit(X_linear[:10000], y_linear[:10000])
#svr = SVR().fit(X_linear[750000:], y_linear[750000:])
#svr.score(X_linear[750000:], y_linear[750000:])
#score = svr.predict(X_linear_test)
#score[df_test.Open == 0] = 0

Очень долго работает и не дает точности лучше, чем линейная регрессия, даже с ядром rbf

<h2>Запись в файл</h2>

In [ ]:
df_out = pd.DataFrame(list(score_final), columns=['Sales'])
df_out = df_out.reset_index().rename(index=str, columns={'index': 'Id'})
df_out.loc[:,'Id'] += 1
df_out.head()

In [ ]:
df_out['Sales'].isnull().unique()

In [ ]:
filepath = 'rossman.csv'
df_out.to_csv(filepath, index=False)

Проверка, что формат правильный:

In [ ]:
filepath = 'rossman.csv'
df_out = pd.read_csv(filepath)
df_out.head()

In [ ]:
len(df_out)

**Вывод:** лучше всего лес, с целевой переменной Customers (число покупателей). <br>Лучшая точность<br>
private: 0.15051<br>
public: 0.12415